## Prognoosi ettevalmistus

- loeme data/ JSON failid
- ühendame nad
- lisame ajadimensioonid
- salvestame tulemuse `data/balance_input.json`

In [1]:
import pandas as pd
from pathlib import Path

In [15]:
project_root = Path("..").resolve()
data_dir = project_root / "data"

consumption_path = data_dir / "raw_consumption.json"
production_path = data_dir / "raw_production.json"
forecast_cons_path = data_dir / "raw_forecast_consumption.json"
forecast_prod_path = data_dir / "raw_forecast_production.json"

consumption_path, production_path, forecast_cons_path, forecast_prod_path

(WindowsPath('C:/Users/tonis/Desktop/Balance-Process-Simulator/data/raw_consumption.json'),
 WindowsPath('C:/Users/tonis/Desktop/Balance-Process-Simulator/data/raw_production.json'),
 WindowsPath('C:/Users/tonis/Desktop/Balance-Process-Simulator/data/raw_forecast_consumption.json'),
 WindowsPath('C:/Users/tonis/Desktop/Balance-Process-Simulator/data/raw_forecast_production.json'))

Impordime JSON failid.

In [16]:
df_consumption = pd.read_json(consumption_path)
df_production = pd.read_json(production_path)
df_forecast_cons = pd.read_json(forecast_cons_path)
df_forecast_prod = pd.read_json(forecast_prod_path)

df_consumption.head(), df_production.head(), df_forecast_cons.head(), df_forecast_prod.head()

(            timestamp  consumption_mwh
 0 2025-12-09 22:00:00           923.18
 1 2025-12-09 22:15:00           929.09
 2 2025-12-09 22:30:00           901.28
 3 2025-12-09 22:45:00           910.34
 4 2025-12-09 23:00:00           860.79,
             timestamp  production_mwh
 0 2025-12-09 22:00:00          440.98
 1 2025-12-09 22:15:00          433.93
 2 2025-12-09 22:30:00          435.96
 3 2025-12-09 22:45:00          427.05
 4 2025-12-09 23:00:00          439.33,
             timestamp  forecast_consumption_mwh
 0 2025-12-09 22:00:00                     823.4
 1 2025-12-09 22:15:00                     817.6
 2 2025-12-09 22:30:00                     807.9
 3 2025-12-09 22:45:00                     823.9
 4 2025-12-09 23:00:00                     760.3,
             timestamp  forecast_production_mwh
 0 2025-12-09 22:00:00                    401.0
 1 2025-12-09 22:15:00                    373.7
 2 2025-12-09 22:30:00                    376.0
 3 2025-12-09 22:45:00               

Ühendame kõik.

In [17]:
df_balance = pd.merge(
    df_consumption,
    df_forecast_cons,
    on="timestamp",
    how="inner"
)

df_balance = pd.merge(
    df_balance,
    df_production,
    on="timestamp",
    how="left"
)

df_balance = pd.merge(
    df_balance,
    df_forecast_prod,
    on="timestamp",
    how="left"
)

df_balance = df_balance.sort_values("timestamp").reset_index(drop=True)
df_balance.head()

,timestamp,consumption_mwh,forecast_consumption_mwh,production_mwh,forecast_production_mwh
0,2025-12-09 22:00:00,923.18,823.4,440.98,401.0
1,2025-12-09 22:15:00,929.09,817.6,433.93,373.7
2,2025-12-09 22:30:00,901.28,807.9,435.96,376.0
3,2025-12-09 22:45:00,910.34,823.9,427.05,377.2
4,2025-12-09 23:00:00,860.79,760.3,439.33,349.0


Lisame ajadimensioonid.

In [18]:
df_balance["timestamp"] = pd.to_datetime(df_balance["timestamp"])

df_balance["date"] = df_balance["timestamp"].dt.date
df_balance["hour"] = df_balance["timestamp"].dt.hour
df_balance["minute"] = df_balance["timestamp"].dt.minute
df_balance["day_of_week"] = df_balance["timestamp"].dt.dayofweek
df_balance["is_weekend"] = df_balance["day_of_week"].isin([5, 6])

df_balance.head()

,timestamp,consumption_mwh,forecast_consumption_mwh,production_mwh,forecast_production_mwh,date,hour,minute,day_of_week,is_weekend
0,2025-12-09 22:00:00,923.18,823.4,440.98,401.0,2025-12-09,22,0,1,False
1,2025-12-09 22:15:00,929.09,817.6,433.93,373.7,2025-12-09,22,15,1,False
2,2025-12-09 22:30:00,901.28,807.9,435.96,376.0,2025-12-09,22,30,1,False
3,2025-12-09 22:45:00,910.34,823.9,427.05,377.2,2025-12-09,22,45,1,False
4,2025-12-09 23:00:00,860.79,760.3,439.33,349.0,2025-12-09,23,0,1,False


Salvestame JSON failina.

In [19]:
balance_input_path = data_dir / "balance_input.json"
df_balance.to_json(balance_input_path, orient="records", date_format="iso")
balance_input_path, len(df_balance)

(WindowsPath('C:/Users/tonis/Desktop/Balance-Process-Simulator/data/balance_input.json'),
 192)

- importisime andmed JSON failidest.
- ühendasime nad ühiseks ajareaks.
- lisasime ajadimensiooni (kuupäev, tund, minut, nädalapäev, nädalavahetus)
- salvestasime tulemuse `data/balance_input.json`.